In [ ]:
# %%
# !pip install lightning timm librosa albumentations torchaudio==2.2.2 audiomentations


In [8]:
# %%
import gc
import logging
import os
import random
import warnings
from datetime import datetime
from pprint import pformat

import albumentations
import librosa
import lightning as L
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import timm
import torch
import torchaudio
import torchmetrics
from joblib import Parallel, delayed
from lightning.pytorch.callbacks import ModelCheckpoint, TQDMProgressBar
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold, train_test_split
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from tqdm.notebook import tqdm

from losses import FocalCosineLoss, FocalLoss, FocalLossBCE, JsdCrossEntropy
from optimizers import Adan, Nadam, NvidiaNovoGrad

sns.set(
    rc={
        "figure.figsize": (8, 4),
        "figure.dpi": 240,
    }
)
sns.set_style("darkgrid", {"axes.grid": False})
sns.set_context("paper", font_scale=0.6)

torch.set_float32_matmul_precision("high")
warnings.simplefilter("ignore")


In [9]:
# %%
class cfg:
    experiment_name = "base_mixup_aug"
    data_path = "../data"

    debug_run = False  # run on a small sample
    experiment_run = False  # run on a stratified data sample
    production_run = True  # run on all data

    normalize_waveform = False # TODO test with and without
    sample_rate = 32000
    n_fft = 2048
    hop_length = 512
    window_length = None
    melspec_hres = 128
    melspec_wres = 312
    freq_min = 40
    freq_max = 15000
    log_scale_power = 2
    max_decibels = 100
    frame_duration = 5
    frame_rate = sample_rate / hop_length

    # vit_b0 = "efficientvit_b0.r224_in1k"
    # vit_b1 = "efficientvit_b1.r224_in1k"
    # vit_b1 = "efficientvit_b1.r288_in1k"
    # vit_b2 = "efficientvit_b2.r224_in1k"
    efnetv2_b0 = "tf_efficientnetv2_b0.in1k"
    # efnetv2s = "tf_efficientnetv2_s.in21k_ft_in1k" # TODO
    # efnetv2_b1 = "tf_efficientnetv2_b1.in1k"
    # vit_m0 = "efficientvit_m0.r224_in1k"
    # vit_m1 = "efficientvit_m1.r224_in1k"
    backbone = efnetv2_b0

    num_classes = 182
    mixup = True
    augment_melspec = True
    add_secondary_labels = False

    label_smoothing = 0.1
    weighted_sampling = False
    sample_weight_factor = 0.25

    accelerator = "gpu"
    precision = "16-mixed"
    n_workers = os.cpu_count() - 2

    n_epochs = 100
    batch_size = 128
    val_ratio = 0.2
    patience = 25

    lr = 1e-3
    lr_min = 1e-6
    weight_decay = 1e-6

    loss = "Focal"
    optimizer = "AdamW"

    timestamp = datetime.now().replace(microsecond=0)
    run_tag = f"{timestamp}_{backbone}_{experiment_name}_val_{val_ratio}_{loss}_{optimizer}_lr_{lr}_decay_{weight_decay}"

    if debug_run:
        run_tag = f"{timestamp}_{backbone}_debug"
        # accelerator = "cpu"
        n_epochs = 5
        batch_size = 32
        num_classes = 10


In [10]:
# %%
def define_logger():
    handlers = [
        logging.StreamHandler(),
        logging.FileHandler(f"../logs/{cfg.run_tag}.log"),
    ]

    if cfg.debug_run:
        handlers = [logging.StreamHandler()]

    logger = logging.getLogger(__name__)
    logging.basicConfig(
        level=logging.INFO,
        format=" %(asctime)s [%(threadName)s] 🐦‍🔥 %(message)s",
        handlers=handlers,
        force=True,  # reconfigure root logger, in case of rerunning -> ensures new file
    )

    return logger


def get_config(cfg) -> None:
    cfg_dictionary = {
        key: value
        for key, value in cfg.__dict__.items()
        if not key.startswith("__") and not callable(key)
    }
    logger.info(f"{'—' * 80}")
    logger.info(f"Config: \n{pformat(cfg_dictionary, indent=1)}")
    return cfg_dictionary


def load_metadata(data_path: str) -> pd.DataFrame:
    logger.info(f"Loading prepared dataframes from {data_path}")
    model_input_df = pd.read_csv(f"{data_path}/model_input_df.csv")
    sample_submission = pd.read_csv(f"{data_path}/sample_submission.csv")

    if cfg.debug_run:
        logger.info("Running debug: sampling data to 10 species and 250 samples")
        top_10_labels = model_input_df["primary_label"].value_counts()[0:10].index
        model_input_df = model_input_df[
            model_input_df["primary_label"].isin(top_10_labels)
        ]
        model_input_df = model_input_df.sample(1000).reset_index(drop=True)

    elif cfg.experiment_run:
        logger.info("Running experiment: sampling data")
        model_input_df = model_input_df.sample(frac=0.1).reset_index(drop=True)

    elif cfg.production_run:
        logger.info("Running production: full data")
        model_input_df = model_input_df.sample(frac=1, random_state=7).reset_index(drop=True)

    logger.info(f"Dataframe shape: {model_input_df.shape}")

    return model_input_df, sample_submission


def read_waveform(filename: str) -> np.ndarray:
    filepath = f"{cfg.data_path}/train_windows_n3/{filename}"
    waveform, _ = librosa.load(filepath, sr=cfg.sample_rate)

    if cfg.normalize_waveform:
        waveform = librosa.util.normalize(waveform)

    return waveform


def read_waveforms_parallel(model_input_df: pd.DataFrame):
    logger.info("Parallel Loading waveforms")
    waveforms = Parallel(n_jobs=cfg.n_workers, prefer="threads")(
        delayed(read_waveform)(filename)
        for filename in tqdm(model_input_df["window_filename"], desc="Loading waves")
    )
    logger.info("Finished loadeding waveforms")
    return waveforms


def create_label_map(submission_df: pd.DataFrame) -> dict:
    logging.info("Creating label mappings")
    cfg.labels = submission_df.columns[1:]
    cfg.num_classes = len(cfg.labels)
    class_to_label_map = dict(zip(cfg.labels, np.arange(cfg.num_classes)))

    return class_to_label_map


def pad_or_crop_waveforms(waveforms: list, pad_method: str = "repeat") -> list:
    logging.info("Padding or cropping waveforms to desired duration")
    desired_length = cfg.sample_rate * cfg.frame_duration

    def _pad_or_crop(waveform: np.ndarray) -> np.ndarray:
        length = len(waveform)

        while length < desired_length:  # repeat if waveform too small
            repeat_length = desired_length - length
            padding_array = waveform[:repeat_length]
            if pad_method != "repeat":
                padding_array = np.zeros(shape=waveform[:repeat_length].shape)
            waveform = np.concatenate([waveform, padding_array])
            length = len(waveform)

        if length > desired_length:  # crop if waveform is too big
            offset = np.random.randint(0, length - desired_length)
            waveform = waveform[offset : offset + desired_length]

        return waveform

    waveforms = [_pad_or_crop(wave) for wave in tqdm(waveforms, desc="Padding waves")]

    return waveforms


def add_sample_weights(
    model_input_df: pd.DataFrame, weight_factor: float = cfg.sample_weight_factor
) -> pd.DataFrame:
    sample_weights = round(
        (
            model_input_df["primary_label"].value_counts()
            / model_input_df["primary_label"].value_counts().sum()
        )
        ** (-weight_factor)
    )
    sample_weights = pd.DataFrame(
        {
            "primary_label": sample_weights.index,
            "sample_weight": sample_weights.values.astype(int),
        }
    )
    model_input_df = model_input_df.merge(
        sample_weights, on="primary_label", how="left"
    )
    return model_input_df


In [19]:
# %%
class BirdDataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        waveforms: list,
        add_secondary_labels: bool = cfg.add_secondary_labels,
        augmentation: list = None,
    ):
        self.df = df
        self.waveforms = waveforms
        self.num_classes = cfg.num_classes
        self.class_to_label_map = class_to_label_map
        self.add_secondary_labels = add_secondary_labels
        self.augmentation = augmentation

        self.mel_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=cfg.sample_rate,
            n_mels=cfg.melspec_hres,
            f_min=cfg.freq_min,
            f_max=cfg.freq_max,
            n_fft=cfg.n_fft,
            hop_length=cfg.hop_length,
            normalized=cfg.normalize_waveform,
            center=True,
            pad_mode="reflect",
            norm="slaney",
            mel_scale="slaney",
        )
        self.db_transform = torchaudio.transforms.AmplitudeToDB(
            stype="power", top_db=cfg.max_decibels
        )

    def create_target(
        self,
        primary_label: str,
        secondary_labels: list,
        secondary_label_weight: float = 1,
    ) -> torch.tensor:
        target = torch.zeros(self.num_classes, dtype=torch.float32)
        # primary_target = torch.tensor(0, dtype=torch.int64)

        if primary_label != "nocall":
            primary_label = self.class_to_label_map[primary_label]
            target[primary_label] = 1
            primary_target = torch.tensor(primary_label, dtype=torch.int64)

            if self.add_secondary_labels:
                secondary_labels = eval(secondary_labels)
                for label in secondary_labels:
                    if label != "" and label in self.class_to_label_map.keys():
                        target[self.class_to_label_map[label]] = secondary_label_weight

        return target, primary_target

    def __len__(self):
        return len(self.waveforms)

    def __getitem__(self, idx):
        waveform = self.waveforms[idx]
        primary_label = self.df.iloc[idx]["primary_label"]
        secondary_labels = self.df.iloc[idx]["secondary_labels"]

        length = len(waveform)
        desired_length = cfg.sample_rate * 5
        if length > desired_length:  # take random 5s crop of 6sec sample
            offset = np.random.randint(0, length - desired_length)
            waveform = waveform[offset : offset + desired_length]

        waveform = torch.tensor(waveform, dtype=torch.float32)

        target, primary_target = self.create_target(
            primary_label=primary_label, secondary_labels=secondary_labels
        )

        melspec = self.db_transform(self.mel_transform(waveform)).to(torch.uint8)
        melspec = melspec.expand(3, -1, -1).permute(1, 2, 0).numpy()

        melspec = melspec - melspec.min()
        melspec = melspec / melspec.max()
        melspec = melspec.astype(np.float32)

        if self.augmentation is not None:
            melspec = self.augmentation(image=melspec)["image"]

        return melspec, target, primary_target


In [20]:
# %%


class GeM(torch.nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = torch.nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        bs, ch, h, w = x.shape
        x = torch.nn.functional.avg_pool2d(
            x.clamp(min=self.eps).pow(self.p), (x.size(-2), x.size(-1))
        ).pow(1.0 / self.p)
        x = x.view(bs, ch)
        return x


class EfficientNetV2(L.LightningModule):
    def __init__(self):
        super().__init__()

        out_indices = (3, 4)
        self.efnetv2 = timm.create_model(
            cfg.backbone,
            features_only=True,
            pretrained=True,
            in_chans=3,
            num_classes=cfg.num_classes,
            out_indices=out_indices,
        )

        feature_dims = self.efnetv2.feature_info.channels()
        logger.info(f"EfficientNetV2 feature dims: {feature_dims}")

        self.global_pools = torch.nn.ModuleList([GeM() for _ in out_indices])
        self.mid_features = np.sum(feature_dims)
        self.neck = torch.nn.BatchNorm1d(self.mid_features)
        self.head = torch.nn.Linear(self.mid_features, cfg.num_classes)

        # self.loss_function = FocalCosineLoss()
        self.loss_function = FocalLossBCE()
        # self.loss_function = FocalLoss()
        # self.loss_function = torch.nn.BCEWithLogitsLoss(reduction="mean")
        # self.loss_function = torch.nn.CrossEntropyLoss(
        #     label_smoothing=cfg.label_smoothing
        # )

        self.accuracy = torchmetrics.Accuracy(
            task="multiclass", num_classes=cfg.num_classes, top_k=1
        )
        self.accuracy_2 = torchmetrics.Accuracy(
            task="multiclass", num_classes=cfg.num_classes, top_k=2
        )
        self.auroc = torchmetrics.AUROC(
            task="multilabel",
            num_labels=cfg.num_classes,
            average="macro",
        )
        self.f1_macro = torchmetrics.F1Score(
            task="multilabel",
            num_labels=cfg.num_classes,
            average="macro",
            threshold=0.5,
        )
        self.f1_weighted = torchmetrics.F1Score(
            task="multilabel",
            num_labels=cfg.num_classes,
            average="weighted",
            threshold=0.5,
        )
        self.lrap = torchmetrics.classification.MultilabelRankingAveragePrecision(
            num_labels=cfg.num_classes,
        )
        self.precision_macro = torchmetrics.classification.MultilabelPrecision(
            num_labels=cfg.num_classes,
            average="macro",
            threshold=0.5,
        )
        self.precision_weighted = torchmetrics.classification.MultilabelPrecision(
            num_labels=cfg.num_classes,
            average="weighted",
            threshold=0.5,
        )
        self.recall_macro = torchmetrics.classification.MultilabelRecall(
            num_labels=cfg.num_classes,
            average="macro",
            threshold=0.5,
        )
        self.recall_weighted = torchmetrics.classification.MultilabelRecall(
            num_labels=cfg.num_classes,
            average="weighted",
            threshold=0.5,
        )

    def normal_mixup(self, melspec, target, alpha=5):
        indices = torch.randperm(melspec.size(0))
        mix_melspec = melspec[indices]
        mix_target = target[indices]

        lam = np.random.beta(alpha, alpha)
        melspec = melspec * lam + mix_melspec * (1 - lam)
        targets = target * lam + mix_target * (1 - lam)

        return melspec, target

    def time_mixup(self, melspec, target):
        indices = torch.randperm(melspec.size(0))
        mix_melspec = melspec[indices]
        mix_target = target[indices]

        melspec = torch.cat([melspec[:, :, :156, :], mix_melspec[:, :, 157:, :]], dim=2)
        target = target + mix_target
        target = torch.clamp(target, max=1)

        return melspec, target

    def freq_mixup(self, melspec, target):
        indices = torch.randperm(melspec.size(0))
        mix_melspec = melspec[indices]
        mix_target = target[indices]

        melspec = torch.cat([melspec[:, :64, :, :], mix_melspec[:, 64:, :, :]], dim=1)
        target = target + mix_target
        target = torch.clamp(target, max=1)

        return melspec, target

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = self.efnetv2(x)
        x = torch.cat(
            [global_pool(m) for m, global_pool in zip(x, self.global_pools)], dim=1
        )
        x = self.neck(x)
        out = self.head(x)

        return out

    def training_step(self, batch, batch_idx):
        x, y, y_primary = batch

        if cfg.mixup:
            choice = np.random.choice(["normal", "none"])
            if choice == "normal":
                x, y = self.normal_mixup(x, y)
            elif choice == "time":
                x, y = self.normal_mixup(x, y)

        y_pred = self(x)
        loss = self.loss_function(y_pred, y)

        y_pred = y_pred.sigmoid()
        y_int = y.to(torch.int64)

        train_accuracy = self.accuracy(y_pred, y_primary)
        train_accuracy_2 = self.accuracy_2(y_pred, y_primary)
        train_f1_weighted = self.f1_weighted(y_pred, y_int)
        train_f1_macro = self.f1_macro(y_pred, y_int)
        train_lrap = self.lrap(y_pred, y_int)

        train_precision_macro = self.precision_macro(y_pred, y_int)
        train_precision_weighted = self.precision_weighted(y_pred, y_int)
        train_recall_macro = self.recall_macro(y_pred, y_int)
        train_recall_weighted = self.recall_weighted(y_pred, y_int)

        self.log(
            "train_loss",
            loss,
            on_step=True,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_accuracy",
            train_accuracy,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_accuracy_2",
            train_accuracy_2,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_f1_weighted",
            train_f1_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_f1_macro",
            train_f1_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_lrap",
            train_lrap,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_precision_macro",
            train_precision_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_precision_weighted",
            train_precision_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_recall_macro",
            train_recall_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "train_recall_weighted",
            train_recall_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )


        return loss

    def validation_step(self, batch, batch_idx):
        x_val, y_val, y_primary_val = batch
        y_pred = self(x_val)
        val_loss = self.loss_function(y_pred, y_val)

        y_pred = y_pred.sigmoid()
        y_val_int = y_val.to(torch.int64)

        val_accuracy = self.accuracy(y_pred, y_primary_val)
        val_accuracy_2 = self.accuracy_2(y_pred, y_primary_val)
        val_auroc = self.auroc(y_pred, y_val_int)
        val_f1_weighted = self.f1_weighted(y_pred, y_val_int)
        val_f1_macro = self.f1_macro(y_pred, y_val_int)
        val_lrap = self.lrap(y_pred, y_val_int)

        val_precision_macro = self.precision_macro(y_pred, y_val_int)
        val_precision_weighted = self.precision_weighted(y_pred, y_val_int)
        val_recall_macro = self.recall_macro(y_pred, y_val_int)
        val_recall_weighted = self.recall_weighted(y_pred, y_val_int)

        self.log(
            "val_loss",
            val_loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_accuracy",
            val_accuracy,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_accuracy_2",
            val_accuracy_2,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_auroc",
            val_auroc,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_f1_weighted",
            val_f1_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_f1_macro",
            val_f1_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_lrap",
            val_lrap,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_precision_macro",
            val_precision_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_precision_weighted",
            val_precision_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_recall_macro",
            val_recall_macro,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )
        self.log(
            "val_recall_weighted",
            val_recall_weighted,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            logger=True,
        )

        return val_loss

    def on_train_epoch_end(self):
        metrics = self.trainer.progress_bar_callback.get_metrics(trainer, model)
        metrics.pop("v_num", None)
        metrics.pop("train_loss_step", None)
        for key, value in metrics.items():
            metrics[key] = round(value, 5)
        logger.info(f"Epoch {self.trainer.current_epoch}: {metrics}")

    def configure_optimizers(self):
        # optimizer = Adan(
        #     model.parameters(),
        #     lr=cfg.lr,
        #     betas=(0.02, 0.08, 0.01),
        #     weight_decay=cfg.weight_decay,
        # )
        optimizer = torch.optim.AdamW(
            params=self.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay,
            fused=False,
        )
        lr_scheduler = CosineAnnealingWarmRestarts(
            optimizer, T_0=cfg.n_epochs, T_mult=1, eta_min=cfg.lr_min, last_epoch=-1
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": lr_scheduler,
                "interval": "epoch",
                "monitor": "train_loss",
                "frequency": 1,
            },
        }


In [21]:
# %%
if __name__ == "__main__":
    logger = define_logger()
    config_dictionary = get_config(cfg)

    csv_logger = None
    if not cfg.debug_run:
        csv_logger = L.pytorch.loggers.CSVLogger(save_dir="../logs/")
        csv_logger.log_hyperparams(config_dictionary)

    model_input_df, sample_submission = load_metadata(data_path=cfg.data_path)
    model_input_df = add_sample_weights(model_input_df)
    class_to_label_map = create_label_map(submission_df=sample_submission)


 2024-06-01 14:29:18,690 [MainThread] 🐦‍🔥 ————————————————————————————————————————————————————————————————————————————————
 2024-06-01 14:29:18,692 [MainThread] 🐦‍🔥 Config: 
{'accelerator': 'gpu',
 'add_secondary_labels': False,
 'augment_melspec': True,
 'backbone': 'tf_efficientnetv2_b0.in1k',
 'batch_size': 128,
 'data_path': '../data',
 'debug_run': False,
 'efnetv2_b0': 'tf_efficientnetv2_b0.in1k',
 'experiment_name': 'base_mixup_aug',
 'experiment_run': False,
 'frame_duration': 5,
 'frame_rate': 62.5,
 'freq_max': 15000,
 'freq_min': 40,
 'hop_length': 512,
 'label_smoothing': 0.1,
 'labels': Index(['asbfly', 'ashdro1', 'ashpri1', 'ashwoo2', 'asikoe2', 'asiope1',
       'aspfly1', 'aspswi1', 'barfly1', 'barswa',
       ...
       'whbwoo2', 'whcbar1', 'whiter2', 'whrmun', 'whtkin2', 'woosan',
       'wynlau1', 'yebbab1', 'yebbul3', 'zitcis1'],
      dtype='object', length=182),
 'log_scale_power': 2,
 'loss': 'Focal',
 'lr': 0.001,
 'lr_min': 1e-06,
 'max_decibels': 100,
 'melsp

In [22]:
# %%
waveforms = read_waveforms_parallel(model_input_df=model_input_df)


 2024-06-01 14:29:19,287 [MainThread] 🐦‍🔥 Parallel Loading waveforms


Loading waves:   0%|          | 0/26592 [00:00<?, ?it/s]

 2024-06-01 14:29:43,979 [MainThread] 🐦‍🔥 Finished loadeding waveforms


In [25]:
# %%
waveforms = pad_or_crop_waveforms(waveforms=waveforms)


 2024-06-01 14:33:26,376 [MainThread] 🐦‍🔥 Padding or cropping waveforms to desired duration


Padding waves:   0%|          | 0/26592 [00:00<?, ?it/s]

In [26]:
    # %%
    train_augmentation = albumentations.OneOf(
        [
            # albumentations.AdvancedBlur(p=0.20),
            # albumentations.GaussNoise(p=0.20),
            # albumentations.ImageCompression(
            #     quality_lower=80, quality_upper=100, p=0.20
            # ),
            albumentations.CoarseDropout(
                max_holes=1, min_height=16, min_width=16, max_height=48, max_width=48, p=0.25
            ),
            albumentations.XYMasking(
                p=0.25,
                num_masks_x=1,
                num_masks_y=1,
                mask_x_length=(3, 20),
                mask_y_length=(3, 20),
            ),
            albumentations.RandomGridShuffle(grid=(2, 2), p=0.25),
            # albumentations.CLAHE(p=0.10),
            # albumentations.Downscale(
            #     scale_min=0.2, scale_max=0.9, interpolation=4, p=0.10
            # ),
            # albumentations.Morphological(p=0.10, scale=(1, 3), operation="erosion"),
            # albumentations.Normalize(p=1),
        ]
    )
    # val_augmentation = albumentations.Compose([albumentations.Normalize(p=1)])

    # grouped split on sample index to keep different windows from the same sample
    # together if splitting randomly this can be considered as a form of leakage
    # validating on a windowed waveform while windows of the same waveform were used for
    # training is easier than classifying a waveform from a different sample, which is
    # the case in practice
    logger.info(f"Splitting {len(waveforms)} waveforms into train/val: {cfg.val_ratio}")
    n_splits = int(round(1 / cfg.val_ratio))
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
    for fold_index, (train_index, val_index) in enumerate(
        kfold.split(
            X=model_input_df,
            y=model_input_df["primary_label"],
        )
    ):
        break

    train_df = model_input_df.iloc[train_index]
    val_df = model_input_df.iloc[val_index]

    train_waveforms = [waveforms[i] for i in train_index]
    val_waveforms = [waveforms[i] for i in val_index]

    train_dataset = BirdDataset(df=train_df, waveforms=train_waveforms, augmentation=train_augmentation)
    val_dataset = BirdDataset(df=val_df, waveforms=val_waveforms)

    train_dataloader = DataLoader(
        train_dataset,
        batch_size=cfg.batch_size,
        drop_last=True,
        num_workers=cfg.n_workers,
        persistent_workers=True,
        pin_memory=True,
    )

    if cfg.weighted_sampling:
        logger.info(
            f"Defining weighted sampling with  factor: {cfg.sample_weight_factor}"
        )
        sample_weight = train_df["sample_weight"].values
        sample_weight = torch.from_numpy(sample_weight)

        weighted_sampler = WeightedRandomSampler(
            sample_weight.type("torch.DoubleTensor"),
            len(sample_weight),
            replacement=True,
        )

        train_dataloader = DataLoader(
            train_dataset,
            batch_size=cfg.batch_size,
            sampler=weighted_sampler,
            drop_last=True,
            num_workers=cfg.n_workers,
            persistent_workers=True,
            pin_memory=True,
        )

    val_dataloader = DataLoader(
        val_dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        drop_last=True,
        num_workers=cfg.n_workers,
        persistent_workers=True,
        pin_memory=True,
    )

    logger.info("Dataloaders ready to go brrr")


 2024-06-01 14:33:29,042 [MainThread] 🐦‍🔥 Splitting 26592 waveforms into train/val: 0.2
 2024-06-01 14:33:29,099 [MainThread] 🐦‍🔥 Dataloaders ready to go brrr


In [27]:
    # %%
    progress_bar = TQDMProgressBar(process_position=1)
    early_stopping = EarlyStopping(
        monitor="train_lrap",
        min_delta=0.01,
        patience=cfg.patience,
        verbose=True,
        mode="max",
    )
    model_checkpoint = ModelCheckpoint(
        monitor="train_lrap",
        every_n_epochs=1,
        mode="max",
        auto_insert_metric_name=True,
        filename=f"{cfg.run_tag}"
        + f"_fold_{fold_index}_"
        + "{epoch}-{val_lrap:.3f}-{val_acc2:.3f}-{val_acc2:.3f}-{val_f1_macro:.3}",
        dirpath="../model_objects/ckpts/"
    )

    os.environ["PJRT_DEVICE"] = "GPU"  # fix for G Cloud to avoid XLA/autocast clash
    model = EfficientNetV2()
    trainer = L.Trainer(
        fast_dev_run=False,
        enable_model_summary=True,
        max_epochs=cfg.n_epochs,
        accelerator=cfg.accelerator,
        precision=cfg.precision,
        callbacks=[progress_bar, early_stopping, model_checkpoint],
        logger=csv_logger,
        log_every_n_steps=10,
    )

    logger.info(f"\nStart training fold {fold_index}")
    trainer.fit(
        model,
        train_dataloaders=train_dataloader,
        val_dataloaders=val_dataloader,
        ckpt_path=None,
    )

    logger.info(f"Finished training fold {fold_index}")
    if not cfg.debug_run and trainer.current_epoch > 10:
        logger.info("Saving model")
        filename = (
            f"{cfg.run_tag}_fold_{fold_index}_epochs_{trainer.current_epoch}.ckpt"
        )
        trainer.save_checkpoint(f"../model_objects/{filename}")
        logger.info(f"Saved model to filename: {filename}")


 2024-06-01 14:33:29,680 [MainThread] 🐦‍🔥 Loading pretrained weights from Hugging Face hub (timm/tf_efficientnetv2_b0.in1k)
 2024-06-01 14:33:29,809 [MainThread] 🐦‍🔥 [timm/tf_efficientnetv2_b0.in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
 2024-06-01 14:33:29,835 [MainThread] 🐦‍🔥 Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
 2024-06-01 14:33:29,837 [MainThread] 🐦‍🔥 EfficientNetV2 feature dims: [112, 192]
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
 2024-06-01 14:33:29,870 [MainThread] 🐦‍🔥 
Start training fold 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

 2024-06-01 14:34:50,396 [MainThread] 🐦‍🔥 Epoch 0: {'val_loss': 0.04834, 'val_accuracy': 0.05888, 'val_accuracy_2': 0.10252, 'val_auroc': 0.23459, 'val_f1_weighted': 0.0, 'val_f1_macro': 0.0, 'val_lrap': 0.14154, 'val_precision_macro': 0.0, 'val_precision_weighted': 0.0, 'val_recall_macro': 0.0, 'val_recall_weighted': 0.0, 'train_loss_epoch': 0.33784, 'train_accuracy': 0.02866, 'train_accuracy_2': 0.05328, 'train_f1_weighted': 0.00943, 'train_f1_macro': 0.00249, 'train_lrap': 0.089, 'train_precision_macro': 0.00144, 'train_precision_weighted': 0.0056, 'train_recall_macro': 0.02953, 'train_recall_weighted': 0.08198}
Metric train_lrap improved. New best score: 0.089


...

 2024-06-01 15:15:40,557 [MainThread] 🐦‍🔥 Epoch 47: {'val_loss': 0.01218, 'val_accuracy': 0.7319, 'val_accuracy_2': 0.80716, 'val_auroc': 0.36704, 'val_f1_weighted': 0.66427, 'val_f1_macro': 0.22274, 'val_lrap': 0.79538, 'val_precision_macro': 0.24094, 'val_precision_weighted': 0.74701, 'val_recall_macro': 0.21792, 'val_recall_weighted': 0.6311, 'train_loss_epoch': 0.00944, 'train_accuracy': 0.73654, 'train_accuracy_2': 0.87312, 'train_f1_weighted': 0.63419, 'train_f1_macro': 0.23552, 'train_lrap': 0.82597, 'train_precision_macro': 0.2424, 'train_precision_weighted': 0.67232, 'train_recall_macro': 0.23548, 'train_recall_weighted': 0.62382}


Validation: |          | 0/? [00:00<?, ?it/s]

 2024-06-01 15:16:33,847 [MainThread] 🐦‍🔥 Epoch 48: {'val_loss': 0.01247, 'val_accuracy': 0.72828, 'val_accuracy_2': 0.80259, 'val_auroc': 0.36614, 'val_f1_weighted': 0.66461, 'val_f1_macro': 0.22216, 'val_lrap': 0.79111, 'val_precision_macro': 0.24089, 'val_precision_weighted': 0.74768, 'val_recall_macro': 0.21665, 'val_recall_weighted': 0.63034, 'train_loss_epoch': 0.00957, 'train_accuracy': 0.72501, 'train_accuracy_2': 0.87199, 'train_f1_weighted': 0.62232, 'train_f1_macro': 0.2333, 'train_lrap': 0.81948, 'train_precision_macro': 0.23937, 'train_precision_weighted': 0.65621, 'train_recall_macro': 0.23372, 'train_recall_weighted': 0.61375}


Validation: |          | 0/? [00:00<?, ?it/s]

 2024-06-01 15:17:27,208 [MainThread] 🐦‍🔥 Epoch 49: {'val_loss': 0.01243, 'val_accuracy': 0.72675, 'val_accuracy_2': 0.80278, 'val_auroc': 0.36637, 'val_f1_weighted': 0.66524, 'val_f1_macro': 0.22225, 'val_lrap': 0.79029, 'val_precision_macro': 0.23977, 'val_precision_weighted': 0.74424, 'val_recall_macro': 0.2178, 'val_recall_weighted': 0.63434, 'train_loss_epoch': 0.00926, 'train_accuracy': 0.74026, 'train_accuracy_2': 0.87345, 'train_f1_weighted': 0.63649, 'train_f1_macro': 0.23878, 'train_lrap': 0.82749, 'train_precision_macro': 0.24493, 'train_precision_weighted': 0.67204, 'train_recall_macro': 0.23876, 'train_recall_weighted': 0.62698}
Monitored metric train_lrap did not improve in the last 25 records. Best score: 0.844. Signaling Trainer to stop.
 2024-06-01 15:17:29,701 [MainThread] 🐦‍🔥 Finished training fold 0
 2024-06-01 15:17:29,702 [MainThread] 🐦‍🔥 Saving model
 2024-06-01 15:17:29,964 [MainThread] 🐦‍🔥 Saved model to filename: 2024-06-01 13:53:58_tf_efficientnetv2_b0.in1k_b